In [1]:
import scanpy as sc
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
os.chdir("/mnt/data/hong/2022/human_placenta")
import matplotlib
custom_params = {"axes.spines.right": False, "axes.spines.top": False}
matplotlib.rcParams['font.family'] = 'sans-serif'
matplotlib.rcParams['font.sans-serif'] = ['Arial']
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42

# matplotlib.rcParams['figure.figsize'] = [5, 5]
# verbosity: errors (0), warnings (1), info (2), hints (3)
sc.settings.verbosity = 3
sc.logging.print_header()
sc.set_figure_params(dpi=300)
%matplotlib inline

adata = sc.read_h5ad('output/10x_h5/h5ad/ad_v2.h5ad')

2024-08-04 17:31:52.277575: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


scanpy==1.9.6 anndata==0.8.0 umap==0.5.5 numpy==1.23.5 scipy==1.11.4 pandas==1.5.3 scikit-learn==1.4.0 statsmodels==0.14.0 igraph==0.10.3 pynndescent==0.5.8


In [9]:
adata = adata[adata.obs['sample']!='placenta_314']

In [10]:
import sccoda
import pertpy as pt
sccoda_model = pt.tl.Sccoda()
sccoda_data = sccoda_model.load(
    adata,
    type="cell_level",
    generate_sample_level=True,
    cell_type_identifier="final_celltypes",
    sample_identifier="sample",
    covariate_obs=["group"],
)
sccoda_data

MuData object with n_obs × n_vars = 37199 × 26929
  2 modalities
    rna:	37188 x 26913
      obs:	'latent_RT_efficiency', 'latent_cell_probability', 'latent_scale', 'doublet_score', 'predicted_doublet', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'n_counts', 'n_genes', 'sample', 'group', 'batch', 'sex', 'mother', 'Hemoglobins', 'S_score', 'G2M_score', 'phase', 'leiden', 'cell type', 'subset', 'C_scANVI', 'maturation_inhouse_three', 'final_celltypes', 'scCODA_sample_id'
      var:	'means', 'variances', 'residual_variances', 'highly_variable_rank', 'highly_variable_nbatches', 'highly_variable_intersection', 'highly_variable'
      uns:	'group_colors', 'hvg', 'leiden', 'neighbors', 'pca', 'pearson_residuals_normalization', 'subset_colors', 'umap'
      obsm:	'X_pca', 'X_umap', 'latent_gene_encoding'
      varm:	'PCs'
      layers:	'log_norm', 'raw', 'sqrtCPMedian', 'sqrt_norm'
      obsp:	'connectivities', 'distances'
    coda:	11 x 16
      obs:	'group', 'sample'
      var:	'n_cells'

In [20]:
abund= abund[:,~np.any(abund.X==0, axis=0)]

In [21]:
abund

View of AnnData object with n_obs × n_vars = 11 × 13
    obs: 'group', 'sample'
    var: 'n_cells'

In [24]:
abund = sccoda_data.mod["coda"]
## find a reference cell type
# from sccoda.util import data_visualization as viz
# viz.rel_abundance_dispersion_plot(
#     data=abund,
#     abundant_threshold=0.9
# )
# plt.show()


In [29]:
# from sccoda.util import comp_ana as mod

model_all = mod.CompositionalAnalysis(
    abund, formula="group", reference_cell_type="pCTB")
all_results = model_all.sample_hmc(num_results=20000)
all_results.set_fdr(est_fdr=0.4)
all_results.summary()

Zero counts encountered in data! Added a pseudocount of 0.5.


100%|██████████| 20000/20000 [01:21<00:00, 245.62it/s]


MCMC sampling finished. (103.201 sec)
Acceptance rate: 52.8%
Compositional Analysis summary:

Data: 11 samples, 16 cell types
Reference index: 15
Formula: group

Intercepts:
                Final Parameter  Expected Sample
Cell Type                                       
B cells                  -0.865        22.028720
CTB                       1.833       327.126531
EVT                      -0.835        22.699594
Endo-1                    1.358       203.435101
Endo-2                   -0.032        50.670660
Fibroblast                1.235       179.890268
Hofbauer cells            1.059       150.859213
Monocyte                 -0.304        38.603658
Myofibroblast             0.458        82.710540
NK cells                 -0.025        51.026599
STB-a                     1.219       177.034927
STB-b                     2.442       601.452186
STB-c                     3.230      1322.589771
T cells                  -0.751        24.688734
eSTB                     -0.278        39.